In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from ipywidgets import interact, FloatSlider

# 1. Generar datos: nube elíptica alargada verticalmente, pero no perfecta
np.random.seed(42)
n = 50
x1 = np.random.normal(0, 0.6, size=n)    # variación moderada en x
x2 = np.random.normal(0, 2.0, size=n)    # mayor variación en y
X = np.column_stack([x1, x2])

# Rotar ligeramente (20°)
theta = np.deg2rad(20)
R = np.array([[np.cos(theta), -np.sin(theta)],
              [np.sin(theta),  np.cos(theta)]])
X = X @ R.T

# Añadir ruido leve
X += np.random.normal(0, 0.05, size=(n, 2))

# 2. PCA
pca = PCA(n_components=2)
pca.fit(X)
components = pca.components_
mean = pca.mean_
explained_var = pca.explained_variance_ratio_

# 3. Visualización interactiva con proyecciones diferenciadas
def plot_rotated(angle_deg=0.0):
    angle_rad = np.deg2rad(angle_deg)
    R_rot = np.array([[np.cos(angle_rad), -np.sin(angle_rad)],
                      [np.sin(angle_rad),  np.cos(angle_rad)]])
    
    # Rotar la nube de puntos
    X_rotated = (X - mean) @ R_rot.T + mean

    fig, ax = plt.subplots(figsize=(7,7))
    ax.set_aspect('equal')
    ax.grid(True, linestyle='--', alpha=0.4)
    ax.set_title(f"Rotación: {angle_deg:.1f}°\n"
                 f"Varianza explicada: PC1 = {explained_var[0]*100:.1f}%, PC2 = {explained_var[1]*100:.1f}%")

    # Puntos rotados
    ax.scatter(X_rotated[:,0], X_rotated[:,1], color='steelblue', alpha=0.6, label='Datos rotados')

    # Ejes X e Y rotados centrados en el gravicentro
    eje_x = R_rot @ np.array([[2], [0]])
    eje_y = R_rot @ np.array([[0], [2]])

    ax.plot([mean[0] - eje_x[0][0], mean[0] + eje_x[0][0]],
            [mean[1] - eje_x[1][0], mean[1] + eje_x[1][0]],
            color='black', linestyle='--', linewidth=1, label='Eje X rotado')

    ax.plot([mean[0] - eje_y[0][0], mean[0] + eje_y[0][0]],
            [mean[1] - eje_y[1][0], mean[1] + eje_y[1][0]],
            color='gray', linestyle='--', linewidth=1, label='Eje Y rotado')

    # Componentes principales (fijas)
    colors_pc = ['crimson', 'seagreen']
    for i in range(2):
        vector = components[i] * 3
        ax.arrow(mean[0], mean[1], vector[0], vector[1], 
                 color=colors_pc[i],
                 width=0.03, head_width=0.2, length_includes_head=True,
                 label=f'PC{i+1}')

    # Proyecciones ortogonales con colores diferenciados
    for point in X_rotated:
        centered = point - mean
        for i in range(2):
            vi = components[i]
            proj = vi @ centered * vi
            proj_point = mean + proj

            # Colores para proyecciones
            color_line = 'darkorange' if i == 0 else 'seagreen'
            color_point = 'darkorange' if i == 0 else 'seagreen'

            ax.plot([point[0], proj_point[0]],
                    [point[1], proj_point[1]],
                    color=color_line,
                    alpha=0.3, linewidth=1)

            ax.scatter(proj_point[0], proj_point[1],
                       color=color_point,
                       alpha=0.5, s=10)

    ax.legend()
    ax.set_xlim(-4, 4)
    ax.set_ylim(-3.5, 3.5)
    plt.show()

# 4. Widget interactivo
interact(plot_rotated, angle_deg=FloatSlider(min=0, max=180, step=1, value=0, description="Rotación"))
